In [1]:
import numpy as np

class PriorityQueue:
    
    def __init__(self, chave=lambda x: x): # Parametrizado com a função usada para comparação. Assim, a classe é mais versátil.
        self.heap = []
        self.chave = chave

    def menor(self):
        if len(self.heap) == 0:
            raise RuntimeError("Tentando ver o menor valor da fila de prioridade vazia")
            
        return self.heap[0]
    def vazia(self):
        if len(self.heap) == 0:
            return True
        else:
            return False
        
    def pop(self):
        if len(self.heap) == 0:
            raise RuntimeError("Tentando extrair valor da fila de prioridade vazia")
            
        menor = self.heap[0]
        self.heap[0] = self.heap[-1]
        self.heap.pop()
        self.desce(0)

        # print("heap: ", self.heap) # debug

        return menor

    def add(self, valor):
        self.heap.append(valor)
        self.sobe(len(self.heap) - 1)
        
        # print("Heap:", self.heap) # debug
    
    def sobe(self, indice):
        if indice == 0: # Já está "no topo"
            return

        pai = (indice - 1) // 2
        if self.chave(self.heap[indice]) < self.chave(self.heap[pai]): # Se o novo elemento for maior que o pai
            tmp = self.heap[pai]
            self.heap[pai] = self.heap[indice]
            self.heap[indice] = tmp
            self.sobe(pai)

    def desce(self, indice):
        indices_filhos = [indice * 2 + 1, indice * 2 + 2]

        if len(self.heap) <= indices_filhos[0]:
            return
        if len(self.heap) == indices_filhos[1]:
            indices_filhos = [indices_filhos[0]]

        filhos = [self.heap[i] for i in indices_filhos]
        
        num_menor_filho = np.argmin(filhos) # Não é o indice na heap, e sim o índice na lista "filhos"
        indice_menor_filho = indices_filhos[num_menor_filho]
        
        if self.chave(self.heap[indice]) > self.chave(self.heap[indice_menor_filho]):    
            tmp = self.heap[indice_menor_filho]
            self.heap[indice_menor_filho] = self.heap[indice]
            self.heap[indice] = tmp
            self.desce(indice_menor_filho)
            

In [2]:
class VAExponencial: # Para modelar o "Inter-Arrival Time" e os tamanhos dos jobs no terceiro cenário
    def __init__(self, Lambda):
        self.Lambda = Lambda

    def amostrar():
        # TODO: Inversa da CDF da exponencial
        pass
        
class VAConstante: # Distribuição dos tamanhos dos jobs no primeiro cenário
    def __init__(self, c):
        self.param = c
    
    def amostrar():
        return self.param    

class VAUniforme: # Distribuição dos tamanhos dos jobs no segundo cenário
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def amostrar():
        # TODO: Inversa da CDF da uniforme
        pass


In [3]:
fila_de_eventos = PriorityQueue()

inter_arrival_time = 2 # Processo de Poisson

fila_s1 = [] # implementar filas
fila_s2 = [] # implementar filas
fila_s3 = [] # implementar filas

params_constante = [0.4, 0.6, 0.95]
params_uniforme = [(0.1, 0.7), (0.1, 1.1), (0.1, 1.8)]
params_exponencial = [0.4, 0.6, 0.95]

# Para colher as métricas em cada situação, descarte os primeiros 10mil jobs que chegarem ("warm up", dando tempo suficiente para
# que o sistema entre em seu estado estacionário), e só então colha as métricas observadas nos próximos 10mil jobs.


### TODO
- Implementar amostragens de VAs
- Implementar filas de jobs
- Pensar em uma estrutura de dados para eventos
- Pensar na estrutura de dados para as métricas de cada job (ID para colher as métricas)